<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/ocp22_load_pretrained_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use OCP22 pretrained models for JARVIS-Leaderboard

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
if not os.path.exists('ocp'):
     !git clone https://github.com/Open-Catalyst-Project/ocp.git
os.chdir('ocp')
!pip install -e .

Cloning into 'ocp'...
remote: Enumerating objects: 11046, done.
remote: Counting objects: 100% (3271/3271), done.
remote: Compressing objects: 100% (1313/1313), done.
remote: Total 11046 (delta 2223), reused 2834 (delta 1921), pack-reused 7775
Receiving objects: 100% (11046/11046), 103.56 MiB | 15.78 MiB/s, done.
Resolving deltas: 100% (7479/7479), done.
Obtaining file:///content/ocp
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ocp-models (pyproject.toml) ... done
  Created wheel for ocp-models: filename=ocp_models-0.0.3-0.editable-py3-none-any.whl size=3457 sha256=8e1f55801c767f1e8e2dd6c0999743aca27f6b208e6d53a76428a2b46241d8b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-on3iykik/wheels/29/db/87/61b8ac706262b22238c01b070b675db289c051f2d890c74f3b
Successfully built ocp-models


In [3]:
pip -q install  lmdb pymatgen==2023.5.10 ase==3.22.1 orjson e3nn==0.2.6 wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.5/367.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
import os, torch
from ase.io import read
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.models import SchNet,GemNetOC

In [10]:
!wget https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_oc22_all_s2ef.pt

--2023-08-29 14:50:17--  https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_oc22_all_s2ef.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.162.83, 108.157.162.35, 108.157.162.120, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.162.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155754987 (149M) [binary/octet-stream]
Saving to: ‘gnoc_oc22_all_s2ef.pt’

gnoc_oc22_all_s2ef. 100%[===================>] 148.54M  20.8MB/s    in 6.5s    

2023-08-29 14:50:25 (23.0 MB/s) - ‘gnoc_oc22_all_s2ef.pt’ saved [155754987/155754987]



In [11]:
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
attrs = torch.load("gnoc_oc22_all_s2ef.pt", map_location=device)["config"][
    "model_attributes"
]

In [12]:
attrs

{'num_spherical': 7,
 'num_radial': 128,
 'num_blocks': 4,
 'emb_size_atom': 256,
 'emb_size_edge': 512,
 'emb_size_trip_in': 64,
 'emb_size_trip_out': 64,
 'emb_size_quad_in': 32,
 'emb_size_quad_out': 32,
 'emb_size_aint_in': 64,
 'emb_size_aint_out': 64,
 'emb_size_rbf': 16,
 'emb_size_cbf': 16,
 'emb_size_sbf': 32,
 'num_before_skip': 2,
 'num_after_skip': 2,
 'num_concat': 1,
 'num_atom': 3,
 'num_output_afteratom': 3,
 'cutoff': 12.0,
 'cutoff_qint': 12.0,
 'cutoff_aeaint': 12.0,
 'cutoff_aint': 12.0,
 'max_neighbors': 30,
 'max_neighbors_qint': 8,
 'max_neighbors_aeaint': 20,
 'max_neighbors_aint': 1000,
 'rbf': {'name': 'gaussian'},
 'envelope': {'name': 'polynomial', 'exponent': 5},
 'cbf': {'name': 'spherical_harmonics'},
 'sbf': {'name': 'legendre_outer'},
 'extensive': True,
 'output_init': 'HeOrthogonal',
 'activation': 'silu',
 'regress_forces': True,
 'direct_forces': True,
 'forces_coupled': False,
 'otf_graph': True,
 'quad_interaction': True,
 'atom_edge_interaction':

In [16]:
model = GemNetOC(None,None,1,**attrs)


state_dict = torch.load("gnoc_oc22_all_s2ef.pt", map_location=device)["state_dict"]
#state_dict = {k[2 * len("module.") :]: v for k, v in state_dict.items()}
model.load_state_dict(state_dict, strict=True)
model.to(device)

RuntimeError: ignored